👀: verifica que si hayas instalado las librerías que vas a necesitar

# Capitulo 2. Proyecto de Machine Learning

En este caso se realizará un modelo para predecir la media de precios en las viviendas de los diferentes municipos de Londres, Reino Unido 🇬🇧.

## <span style="color:green">1. Descargar los datos</span>

Las bases de datos para este proyecto se puede encontrar en este enlace: https://www.kaggle.com/justinas/housing-in-london

También se pueden consultar todas las bases de datos de este curso en GitHub:https://github.com/a2Proyectos/MachineLearning_Data

- housing_in_london_yearly_variables.csv, con los datos que necesitamos para hacer la regresión.
- London_Borough_Excluding_MHW.shp, con los datos que necesitamos para graficar Londres.
- Capitulo_2/housing_in_london_monthly_variables.csv, con los datos de la media salarial

In [ ]:
## Importamos nuestras librerias principales panda, numpy, matplotlib, os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
## Definimos una función para extraer datos.
#DOWNLOAD_ROOT es la base del GitHub donde vamos a estar descargando las bases de datos.
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ANFESADI15/A2/main/C2/"
#Complementos con la dirección especifica de la base de datos que queremos.
LONDON_SALARY = "MachineLearning_Data-main/MachineLearning_Data-main/Capitulo_2/housing_in_london_yearly_variables.csv"
LONDON_HOUSING = "MachineLearning_Data-main/MachineLearning_Data-main/Capitulo_2/housing_in_london_monthly_variables.csv"
LONDON_MAP = os.path.abspath("") + "\map\London_Borough_Excluding_MHW.shp"


def extraer_datos(root,database):
    csv_path = root + database
    return pd.read_csv(csv_path)


***

## <span style="color:green">2. Vistazo a la Base de Datos</span>

In [ ]:
## Juntamos nuestra base de datos de la media salarial, con la de datos de Londres.
df1 = extraer_datos(DOWNLOAD_ROOT, LONDON_SALARY)
df2 = extraer_datos(DOWNLOAD_ROOT, LONDON_HOUSING)
df1.head()

In [ ]:
#Filtrar los datos 
df1 = df1.filter(items=["median_salary","area","date"])
df1.head()

In [ ]:
#Fusionar los dos dataframe
data = pd.merge(df1,df2)
data.head()

In [ ]:
# Obtener información de los datos.
"population_size","life_satisfaction","number_of_houses"
data.info()

👀:Comienza a ver el tipo de variable, de eso depende el análisis que le demos o si necesitamos cambiarlas a otro tipo de variable

***

In [ ]:
# Una forma muy común para saber que tipo de datos contiene alguna variable de tipo objeto, es contar sus valores, ejemplo:
data["area"].value_counts()

In [ ]:
# Una forma muy utilizada para obtener información de nuestros datos numéricos es el método describe(), ejemplo:
pd.options.display.float_format = '{:,.2f}'.format
data.describe()

In [ ]:
# Realiza un Histograma para visualizar los datos 
#matplotlib inline para aquellos que estan con jupyter notebook
data.hist(bins=50,figsize=(15,10))
plt.show()

***

## <span style="color:green">3. Crear un set de entrenamiento y de prueba </span>

Recuerden que nuestro set de prueba lo pondremos de lado por ahora, sin verlo. No sean tramposos ❌

In [ ]:
#Importamos la función para dividir los datos train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Seleccionamos por ejemplo, el 30% de los datos para el set de prueba
#Random_state es la semilla que se usa para generar números aleatorios.
set_ent, set_prueba = train_test_split(data, test_size=0.3, random_state=45)

In [ ]:
#confirmamos la división
print(len(set_ent),len(set_prueba))

Ya tenemos nuestro set de prueba y nuestro set de entrenamiento ✅

***

### <span style="color:blue">3.1 Evitar Sesgo </span>

In [ ]:
# Para categorizar una variable, ejemplo con 5 niveles. 
data["salary_cat"] = pd.cut(data["median_salary"],
                           bins=[0., 10000, 20000, 30000, 40000,
                                np.inf],
                           labels=[1, 2, 3, 4, 5])
#Hacer un histograma de las categorías
data["salary_cat"].hist()

In [ ]:
#Verificar que no existen datos en el bin #1
data["salary_cat"].value_counts()

In [ ]:
# Redefinir en 4 categorías, quitando en donde no hay datos (el bin#1)
data = data.dropna(subset=['median_salary'])
data = data.reset_index()
data["salary_cat"] = pd.cut(data["median_salary"],
                           bins=[10000., 20000., 30000., 40000.,
                                np.inf],
                           labels=[1, 2, 3, 4])
data["salary_cat"].hist()

In [ ]:
#Verifica que si se redefinieron las categorías 
data["salary_cat"].value_counts()

In [ ]:
## Dividir datos basándonos en nuestras categorías de salarios

from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# Generamos nuestro objeto para que lo divida en 30% y solo haga una división
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=45)

In [ ]:
# Creamos nuestras variables basándonos en nuestras categorías
for ent_index, prueba_index in split.split(data,data["salary_cat"]):
    cat_set_ent = data.loc[ent_index]
    cat_set_prueba = data.loc[prueba_index]

In [ ]:
# Comprobación. Ya en porcentaje
cat_set_prueba["salary_cat"].value_counts()/ len(cat_set_prueba)

***

In [ ]:
# Creamos dataframe para trabajar con el set de entrenamiento
df = cat_set_ent.copy()
df.head()

***

## <span style="color:green">4. Visualizar los Datos Gráficamente </span>


Para esto vamos a necesitar, en conjunto con nuestro dataset LONDON_MAP, una nueva libreria que se llama geopandas, el cual exteiende la libreria pandas, para trabajar con datos geoespaciales, se puede encontrar más información en: https://geopandas.org/getting_started/introduction.html

In [ ]:
#Importar geopandas
import geopandas as gpd

#Leer el mapa
londres_map = gpd.read_file(LONDON_MAP)
londres_map.head()

In [ ]:
# Graficamos el mapa
londres_map.plot()

***

In [ ]:
# Ajustamos los nombres de las columnas para despúes hacer un merge.
#utilizar lower para cambiar de mayúsculas a minúsculas 
londres_map.columns = londres_map.columns.str.lower()
londres_map = londres_map.rename({'name': 'area', 'gss_code': 'code'}, axis=1)
londres_map["area"] = londres_map["area"].str.lower()


#Seleccionar columnas necesarias
londres_map = londres_map.filter(items=["area","code","hectares","geometry"])
londres_map.head()

***

In [ ]:
# Seleccionar datos de nuestro set de entrenamiento
df_m = df.groupby('area').agg({'average_price': ['mean'], 'houses_sold': 'sum'})

# Le asignamos nombre a las columnas del nuevo dataframe y reseteamos el indice
df_m.columns = ['average_price', 'houses_sold']
df_m.reset_index(inplace = True)
df_m.head()

In [ ]:
# Combinar dtaframes
londres_map = pd.merge(londres_map,df_m,on="area")
londres_map.head()

***

In [ ]:
#Gráfica del promedio de los precios en las casas 
#Cuando se grafica en geopandas hay muchos argumentos, no se desesperen si no los recuerdan, es normal. 
plt = londres_map.plot(column = 'average_price', cmap = 'Reds', edgecolor = 'maroon',
               legend = True, legend_kwds = {'label': 'Precio', 'orientation' : 'horizontal'})
plt.set_title('Media de los precios en las casas')
plt.axis('off')

💸: Recuerden que los precios son mayores en el centro de la cuidad

In [ ]:
#Graficar ahora el total de las casa vendidas (utilizar el código anterior para no repetir)
plt = londres_map.plot(column = 'houses_sold', cmap = 'Blues', edgecolor = 'maroon',
               legend = True, legend_kwds = {'label': 'Precio', 'orientation' : 'horizontal'})
plt.set_title('Total de casas vendidas')
plt.axis('off')

**Ejercicio:** Incluir en el análisis los salarios promedio por zona (apoyarse del video 21)

***

## <span style="color:green">5. Medir la Correlación </span>


In [ ]:
# Crear matriz de correlación
matriz = df.corr(method='pearson')

# Comparar correlación
matriz["average_price"].sort_values(ascending=False)

In [ ]:
#Importar seaborn 
import seaborn as sns

# Crear vector
mask = np.triu(np.ones_like(matriz, dtype = bool))

# Graficar
plt = sns.heatmap(matriz, mask = mask, annot = True, cmap = 'YlGnBu_r')
plt

In [ ]:
#Importar pandas 
from pandas.plotting import scatter_matrix

#Para graficar scatter_matrix...
columns = ['average_price', 'median_salary', 'no_of_crimes', 'houses_sold']
scatter_matrix(df[columns], figsize = (12, 12), color = '#D52B06', alpha = 0.3, 
               hist_kwds = {'color':['bisque'], 'edgecolor': 'firebrick'});


In [ ]:
#Por si quieren ver una gráfica en específico más a detalle
df.plot(kind="scatter",y="median_salary",x="average_price",alpha=0.1)

***

## <span style="color:green">6. Combinación de Variables </span>


In [ ]:
#Armar una columna para hacer las combinaciones que necesitamos
df["vendidas_poblacion"] = df["population_size"] / df["houses_sold"]
# Crear matriz de correlación
matriz = df.corr(method='pearson')
matriz["average_price"].sort_values(ascending=False)

***

***

## <span style="color:Blue">7. Transformación de Datos </span>


In [ ]:
# Crear Dataframe de predictores y variable a predecir ✂️
df_label = cat_set_ent['average_price']
#Nuevo dataframe sin average_price
df = cat_set_ent.drop('average_price', axis=1)
df.head()

***

In [ ]:
#Darte una idea de a qué variables le hace faltan datos
df.info()
#Calcular la suma de todos los vacíos  
df.isna().sum()

In [ ]:
#Quitar el número de crímenes por su alta cantidad de datos vacíos 
df=df.drop("no_of_crimes", axis=1) 

In [ ]:
#Tomamos la media 
median = df["houses_sold"].median()
#Llenamos los valores con la media
df['houses_sold'].fillna(median,inplace=True)  #opción 3
#Verifica que no hay datos vacíos
df.isna().sum()

***

Recuerden que volvimos a ejecutar la línea de código de "df = cat_set_ent.drop('average_price', axis=1)" para hacer este ejercicio

In [ ]:
#BIENVENIDOS A SCIKIT <3 Serán unos expertos al final. 
# Rellenar valores con scikit
#1. Recuerden: importar lo que vayas a utiliza, en este caso SimpleImputer
from sklearn.impute import SimpleImputer

# 2. Crear objeto, en este caso se llamará imputer
imputer = SimpleImputer(strategy="median")

df.isna().sum()
df= df.drop(["no_of_crimes"],axis=1)
df.info()

# data numérico
df_num = df.drop(["area","date","code"],axis=1)

#Ejecuta Imputer
imputer.fit(df_num)

#Aplicar transform para rellenar las medianas 
X = imputer.transform(df_num)

#Regresarlo a dataframe 
df_tr = pd.DataFrame(X, columns=df_num.columns, index=df_num.index)
df_tr.head()

***

## <span style="color:Blue">8. Manejo de texto y valores categóricos </span>

In [ ]:
#definir que variable vamos a cambiar a valor numérico
df_cat=df[["area"]]
#Convertir variables de texto en numéricas
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
df_oe = ordinal_encoder.fit_transform(df_cat)
#verificar que si se haya transformado
df_oe[0:10]

#Por si quieres ver como funciona el encoder
ordinal.encoder.categories_

In [ ]:
#ONE HOT ENCODER, es importante recordar y entender esta función porque la usaremos en todo el curso
#Convertir variables categóricas en binarias
#Importar OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
#Crear objeto
encoder = OneHotEncoder()
#Ajustar
df_1hot = encoder.fit_transform(df_cat)
#obligar a que nos muestre la matriz 
df_1hot.toarray()

***

## <span style="color:Blue">9. Escalación de variables </span>


#### <span style="color:Blue">9.1 Normalización </span>

In [ ]:
#Importar MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
#Creamos el objeto 
scaler = MinMaxScaler()
#Hacer un dataframe con la transformación
pd.DataFrame(scaler.fit_transform(prueba),columns=prueba.columns,index=prueba.index)

#### <span style="color:Blue">9.2 Estandarización </span>

In [ ]:
## Estandarización (Es el más ultizado)
#Importar StandardScaler (presten especial atención a StandardScaler lo estaremos viendo muy seguido) 
from sklearn.preprocessing import StandardScaler
#Crea objeto
scaler = StandardScaler()
#Hacer un dataframe con la transformación
pd.DataFrame(scaler.fit_transform(prueba),columns=prueba.columns,index=prueba.index)

***

## <span style="color:Blue">10. Pipeline </span>


In [ ]:
# Crear función de pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([("remover",RemoverOutliers()),
                      ("rellenar",SimpleImputer(strategy="median")),
                       ("escalar",StandardScaler())])
pd.DataFrame(pipeline.fit_transform(prueba),columns=prueba.columns,index=prueba.index)

In [ ]:
# Column Transformer (nos permite transformar varias columnas al mismo tiempo y luego juntarlas)
from sklearn.compose import ColumnTransformer
num = list(df_num)
cat = ["area"]

pipeline_completo = ColumnTransformer([
    ("num", pipeline, num),
    ("cat", OneHotEncoder(), cat)
])
#Hacer un datframe que usaremos para la regresión linea
df_preparado = pipeline_completo.fit_transform(df)
#Visualiza los datos 
df_preparado

***

***

## <span style="color:purple">11. Seleccionar y entrenar modelos </span>

In [ ]:
## Hacer Regresión Lineal (AL FIN)
from sklearn.linear_model import LinearRegression
reg_lin = LinearRegression()
reg_lin.fit(df_preparado, df_label)

algunos_datos = df.iloc[:5]
datos_predecir = df_label.iloc[:5]
datos_transformados = pipeline_completo.transform(algunos_datos)
print("Predicción:",reg_lin.predict(datos_transformados))
print("\nOriginales:",list(datos_predecir))

In [ ]:
#Análisis preambultario de los errores
algunos_datos=df.iloc[:5]
dato_predecir=df_label.iloc[:5]
datos_transformados=pipeline_completo.transform(algunos_datos)

vp=list(reg_lin.predict(datos_transformados))
vr=list(dato_predecir)

vp=pd.Series(vp)
vr=pd.Series(vr)


abs((vr-vp)/vr).mean()

***

#### <span style="color:purple">11.1 RMSE </span>

In [ ]:
# Calcular el promedio de la suma de los errores al cuadrado RMSE
from sklearn.metrics import mean_squared_error
prediccion = reg_lin.predict(df_preparado)
error = mean_squared_error(df_label,prediccion)
error = np.sqrt(error)
error

In [ ]:
#Sacar el promedio 
df_label.mean()

In [ ]:
#Calcular el porcentaje de acierto
error/df.label.mean()

***

#### <span style="color:purple">11.2 Árbol de Decisión </span>

In [ ]:
# importar DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
#crear objeto 
reg_arbol = DecisionTreeRegressor()
#correrlo
reg_arbol.fit(df_preparado,df_label)
prediccion = reg_arbol.predict(df_preparado)
#calcular el error
error = mean_squared_error(df_label,prediccion)
error = np.sqrt(error)
error

***

#### <span style="color:purple">11.3 Validación Cruzada </span>

¡Anota esto porque es importante!

In [ ]:
#Importar cross_val_score, creo objeto, corro mi función
from sklearn.model_selection import cross_val_score
resultados = cross_val_score(reg_arbol, df_preparado, df_label, scoring="neg_mean_squared_error",cv = 10)
rmse = np.sqrt(-resultados)
rmse

In [ ]:
#Calcular el porcentaje de acierto de validación cruzada 
rmse.mean()/df.label.mean()

***

#### <span style="color:purple">11.4 Bosque Aleatorio </span>
¡Este también es importante!

In [ ]:
#Importar RandomForestRegressor, creo objeto, corro mi función
from sklearn.ensemble import RandomForestRegressor
reg_forest = RandomForestRegressor()
reg_forest.fit(df_preparado,df_label)
prediccion = reg_forest.predict(df_preparado)

#calcular el error
error = mean_squared_error(df_label,prediccion)
error = np.sqrt(error)
error

In [ ]:
#Calcular el el promedio de la suma de los errores al cuadrado
resultados = cross_val_score(reg_forest, df_preparado, df_label, scoring="neg_mean_squared_error",cv = 10)
rmse = np.sqrt(-resultados)
rmse

In [ ]:
#Calcular el porcentaje de acierto de bosque aleatorio
rmse.mean()/df.label.mean()

***

## <span style="color:purple">12. Afinar el modelo </span>

### <span style="color:purple">12.1 Grid Search</span>

In [ ]:
#Importar GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = [{
    'n_estimators': [3,10,30], 'max_features': [2,4,6,8]
}]
grid_search = GridSearchCV(reg_forest,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(df_preparado,df_label)
#Calcular el mejor parámetro
grid_search.best_params_
#Ver el error
np.sqrt(-grid_search.best_score_)

***

### <span style="color:purple">12.2 set de prueba</span>

In [ ]:
#¡AHORA SI! A utilizar el set de prueba. Es la última parte 

#Define cuál es el modelo final 
modelo_final = grid_search.best_estimator_

#Aquí definimos cuál será nuestra variable a predecir y las predichas 
Y = cat_set_prueba["average_price"].copy()
X = cat_set_prueba.drop("average_price",axis=1)

#Ahora, sí. El pipeline para limpieza de datos 
X_preparada = pipeline_completo.transform(X)
prediccion_final = modelo_final.predict(X_preparada)

#Por último, ver el error del modelo 
mse_final = mean_squared_error(Y, prediccion_final)
rmse = np.sqrt(-mse_final)
np.sqrt(mse_final)

In [ ]:
#Ver el porcentaje de acierto
rmse/Y.mean()